# 파이프라인 만들기

이전 랩에서는 Azure Machine Learning SDK를 사용하여 데이터 액세스, 학습 실험 실행, 기계 학습 모델 등록 등의 전체 모델 학습 프로세스를 수행하는 방법을 살펴보았습니다. 지금까지는 기계 학습 솔루션을 대화형으로 만드는 데 필요한 여러 단계를 수행했습니다. 이 랩에서는 *파이프라인*을 사용하여 이러한 단계를 자동화하는 방법을 살펴봅니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험용 데이터 준비

이 랩에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래의 셀을 실행하여 이 데이터 세트를 만듭니다(이전 랩에서 데이터 세트를 만들었다면 코드가 기존 버전을 찾습니다).

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                       target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                       overwrite=True, # 이름이 같은 기존 파일 바꾸기
                       show_progress=True)

    # 데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 테이블 형식 데이터 세트 등록
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 파이프라인 단계용 스크립트 만들기

파이프라인은 *단계* 하나 이상으로 구성됩니다. 이러한 단계는 Python 스크립트일 수도 있고, 특정 위치 간에 데이터를 복사하는 데이터 전송 단계나 자동 ML 학습 추정기 등의 특수 단계일 수도 있습니다. 각 단계는 자체 컴퓨팅 컨텍스트에서 실행할 수 있습니다.

이 연습에서는 모델 학습용 추정기 단계와 학습된 모델 등록용 Python 스크립트 단계가 포함된 간단한 파이프라인을 작성합니다.

In [ ]:
import os
# 파이프라인 단계 파일용 폴더 만들기
experiment_folder = 'diabetes_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

이제 첫 단계(모델 학습)용 스크립트를 만들 수 있습니다. 이 스크립트에 포함된 **output_folder** 매개 변수는 학습된 모델을 저장해야 하는 폴더를 참조합니다.

In [ ]:
%%writefile $experiment_folder/train_diabetes.py
# 라이브러리 가져오기
from azureml.core import Run
import argparse
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 매개 변수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="diabetes_model", help='output folder')
args = parser.parse_args()
output_folder = args.output_folder

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['diabetes_train'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# 학습된 모델 저장
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

파이프라인의 두 번째 단계용 스크립트는 저장된 위치에서 모델을 로드한 다음 작업 영역에 등록합니다. 이 스크립트에는 모델이 저장된 경로를 포함하는 **model_folder** 매개 변수 하나가 포함됩니다.

In [ ]:
%%writefile $experiment_folder/register_diabetes.py
# 라이브러리 가져오기
import argparse
import joblib
from azureml.core import Workspace, Model, Run

# 매개 변수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--model_folder', type=str, dest='model_folder', default="diabetes_model", help='model location')
args = parser.parse_args()
model_folder = args.model_folder

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 모델 로드
print("Loading model from " + model_folder)
model_file = model_folder + "/model.pkl"
model = joblib.load(model_file)

Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'})

run.complete()

## 파이프라인 단계용 컴퓨팅 환경 준비

이 연습에서는 두 단계에 같은 컴퓨팅을 사용하지만 각 단계는 독립적으로 실행됩니다. 따라서 해당하는 경우 각 단계에 서로 다른 컴퓨팅 컨텍스트를 지정할 수 있습니다.

먼저 컴퓨팅 대상을 가져옵니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# 클러스터가 있는지 확인
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 클러스터가 없으면 생성
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS1_V2', 
                                                           max_nodes=2)
    pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

컴퓨팅에는 필요한 패키지 종속성이 설치된 Python 환경이 필요하므로 실행 구성을 만들어야 합니다.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# 실험용 Python 환경 만들기
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Azure ML의 종속성 관리 허용
diabetes_env.docker.enabled = True # Docker 컨테이너 사용

# 패키지 종속성 집합 만들기
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                                             pip_packages=['azureml-sdk','pyarrow'])

# 환경에 종속성 추가
diabetes_env.python.conda_dependencies = diabetes_packages

# 환경 등록(이전 랩을 완료하지 않은 경우에 한함)
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# 파이프라인용 새 runconfig 개체 만들기
pipeline_run_config = RunConfiguration()

# 위에서 만든 컴퓨팅을 사용합니다. 
pipeline_run_config.target = pipeline_cluster

# 실행 구성에 환경 할당
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

## 파이프라인 작성 및 실행

이제 파이프라인을 만들고 실행할 수 있습니다.

먼저 파이프라인용 단계와 파이프라인 간에 전달해야 하는 데이터 참조를 정의해야 합니다. 이 연습의 첫 번째 단계는 두 번째 단계에서 읽을 수 있는 폴더에 모델을 써야 합니다. 이 두 단계는 원격 컴퓨팅에서 실행되며 각기 다른 컴퓨팅에서 실행할 수 있으므로, 작업 영역 내 데이터 저장소의 특정 위치에 대한 데이터 참조로 폴더 경로를 전달해야 합니다. **PipelineData** 개체는 중간 스토리지 위치에 사용되며 파이프라인 단계 간에 전달할 수 있는 특수한 종류의 데이터 참조입니다. 여기서는 PipelineData 개체를 만들어 첫 번째 단계의 출력/두 번째 단계의 입력으로 사용할 것입니다. 또한 데이터 참조를 통해 참조하는 데이터 저장소 위치에 코드가 액세스할 수 있도록 이 개체를 스크립트 인수로 전달해야 합니다.

In [ ]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.estimator import Estimator

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 모델 폴더용 PipelineData(임시 데이터 참조) 만들기
model_folder = PipelineData("model_folder", datastore=ws.get_default_datastore())

estimator = Estimator(source_directory=experiment_folder,
                        compute_target = pipeline_cluster,
                        environment_definition=pipeline_run_config.environment,
                        entry_script='train_diabetes.py')

train_step = EstimatorStep(name = "Train Model",
                           estimator=estimator, 
                           estimator_entry_script_arguments=['--output_folder', model_folder],
                           inputs=[diabetes_ds.as_named_input('diabetes_train')],
                           outputs=[model_folder],
                           compute_target = pipeline_cluster,
                           allow_reuse = True)

# 2단계: 모델 등록 스크립트 실행
register_step = PythonScriptStep(name = "Register Model",
                                source_directory = experiment_folder,
                                script_name = "register_diabetes.py",
                                arguments = ['--model_folder', model_folder],
                                inputs=[model_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

이제 정의한 단계에서 파이프라인을 작성하여 실험으로 실행할 준비가 되었습니다.

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# 파이프라인 생성
pipeline_steps = [train_step, register_step]
pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

# 실험 작성 및 파이프라인 실행
experiment = Experiment(workspace = ws, name = 'diabetes-training-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

위의 위젯에는 실행 중인 파이프라인의 세부 정보가 표시되어 있습니다. [Azure Machine Learning Studio](https://ml.azure.com)의 **실험** 페이지에서 파이프라인 실행을 모니터링할 수도 있습니다.

파이프라인 실행이 완료되면 *학습 컨텍스트* 태그가 지정된 새 모델이 등록됩니다. 이 태그는 해당 모델이 파이프라인에서 학습되었음을 나타냅니다. 다음 코드를 사용하여 모델 등록 여부를 확인합니다.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

이 연습은 파이프라인 작성 원칙을 보여 주는 간단한 예제입니다. 실제로는 더 복잡한 논리를 작성하여 파이프라인 단계에 포함할 수 있습니다. 예를 들어 특정 테스트 데이터를 기준으로 모델을 평가해 AUC, 정확도 등의 성능 메트릭을 계산하고, 이 메트릭을 이전에 등록한 모델 버전의 메트릭과 비교한 다음 성능이 더 우수한 경우에만 새 모델을 등록할 수 있습니다.

[Azure DevOps용 Azure Machine Learning 확장](https://marketplace.visualstudio.com/items?itemName=ms-air-aiagility.vss-services-azureml)을 사용하여 Azure ML 파이프라인을 Azure DevOps 파이프라인과 결합한 다음 *CI/CD(연속 통합/지속적인 배포)* 프로세스에 모델 재학습 과정을 통합할 수 있습니다. 이 두 파이프라인은 이름이 같아* *혼동할 수 있으므로 주의하세요. 예를 들어 Azure DevOps *빌드* 파이프라인을 사용해 모델 학습과 등록을 수행하는 Azure ML 파이프라인을 트리거할 수 있습니다. 모델이 등록되면 빌드 파이프라인은 Azure DevOps *릴리스* 파이프라인을 트리거할 수 있습니다. 릴리스 파이프라인은 모델을 사용하는 애플리케이션이나 서비스와 함께 모델을 웹 서비스로 배포합니다.